# Politics in Nevada

In [1]:
SCHOOL="nevada"
SUBJECT="opinion"
START_YEAR=2013
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [3]:
%cd ..

/home/alexandersuen/diversity-colleges


In [4]:
%cd src

/home/alexandersuen/diversity-colleges/src


In [5]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [6]:
%cd ..

/home/alexandersuen/diversity-colleges


In [7]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [8]:
df.shape

(930, 6)

In [9]:
df.head(2)

,title,url,body,year,month,day
0,"Sorry underage friends, your “Old Ironsides” d...",https://archive.nevadasagebrush.com/blog/2022/...,"Sorry underage friends, your “Old Ironsides” d...",2022,5,8
1,You are not alone: Mental health awareness month,https://archive.nevadasagebrush.com/blog/2022/...,You are not alone: Mental health awareness mon...,2022,5,6


In [10]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")
#BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
print(BIPARTISAN_API_KEY)
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g==


In [11]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

14.256245


In [12]:
df['year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013])

In [13]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2013, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2013 ... 33
2013	33	119.00817499999998	3.606308333333333
Year: 2014 ... 195
2014	189	331.04381900000016	1.7515545978835987
Year: 2015 ... 175
2015	171	146.83804399999985	0.8587020116959055
Year: 2016 ... 147
2016	143	-79.16155599999995	-0.5535773146853143
Year: 2017 ... 92
2017	92	-36.619383000000006	-0.3980367717391305
Year: 2018 ... 98
2018	95	-58.588921	-0.6167254842105263
Year: 2019 ... 103
2019	102	-95.85229000000001	-0.9397283333333334
Year: 2020 ... 61
2020	60	-61.298336	-1.0216389333333333
Year: 2021 ... 19
2021	19	-53.998701999999994	-2.842036947368421
Year: 2022 ... 7
2022	7	11.752118000000001	1.6788740000000002


In [14]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)

In [ ]:
# ASYNCHRONOUS METHOD : should be a lot faster
import asyncio

results = []

for year in range(2013, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2013 ... 33
2013	33	119.00817499999998	3.606308333333333
Year: 2014 ... 195
2014	193	324.78547300000025	1.6828262849740945
Year: 2015 ... 175
